In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
from nbodykit.lab import *
from nbodykit import setup_logging, style

import os
import matplotlib.pyplot as plt
plt.style.use(style.notebook)

import numpy as np

/opt/anaconda2/envs/nbodykit-env/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
setup_logging() # turn on logging to screen

In [4]:
redshift = 0.5
cosmo = cosmology.Planck15

In [5]:
# define download dir for the data to the current directory
download_dir = "."
#download_dir = "/Users/mar/Research/Codigos/NagBody_pkg/NagBody_sources/colas/mglpicola_fofr/runs/mglpicola_fofr/B0256_P0256_M0256_testing/F5/box5001/output/."
#download_dir = "/Users/mar/Research/Codigos/NagBody_pkg/NagBody_sources/colas/mglpicola_fofr/runs/mglpicola_fofr/B0256_P0256_M0256/F5/box5001/output/."
#download_dir = "./gadget_data_27"

In [6]:
# NOTE: change this path if you downloaded the data somewhere else!
data_path = os.path.join(download_dir, 'snapshot_z0p000.0')
#data_path = os.path.join(download_dir, 'snapshot_z0p000.*')
#data_path = os.path.join(download_dir, 'gadget.*')


# initialize the Gadget1 catalog objects for data and randoms
data = Gadget1Catalog(data_path)

NameError: global name 'FileNotFoundError' is not defined

We can analyze the available columns in the catalogs via the ``columns`` attribute:

In [ ]:
print('data columns = ', data.columns)

In [ ]:
# convert to a MeshSource, using TSC interpolation on 256^3 mesh:: tsc or cic
#mesh = data.to_mesh(resampler='tsc', Nmesh=512, compensated=True, position='Position')
#mesh = data.to_mesh(resampler='tsc', Nmesh=1024, compensated=True, position='Position')
mesh = data.to_mesh(resampler='tsc', Nmesh=1024, compensated=True, position='Position',interlaced=True)

In [ ]:
print("mesh = ", mesh)

In [ ]:
# compute the power, specifying desired linear k-binning
#r = FFTPower(mesh, mode='1d', dk=0.005, kmin=0.01)
r = FFTPower(mesh, mode='1d', dk=0.01, kmin=0.001)

In [ ]:
# the result is stored at "power" attribute
Pk = r.power
print(Pk)

In [ ]:
print(Pk.coords)

In [ ]:
kvec=Pk['k']
pkvec=Pk['power'].real

#print(kvec[3])
#print(pkvec[3])

nk = kvec.shape[0]

print(nk)

pofka=np.array([kvec,pkvec])
pofkb=np.transpose(pofka)
print pofkb

np.savetxt('pofk.txt',pofkb,delimiter='\t',newline='\r\n')

In [ ]:
# print out the meta-data
for k in Pk.attrs:
    print("%s = %s" %(k, str(Pk.attrs[k])))

In [ ]:
fig1 = plt.figure()

# print the shot noise subtracted P(k)
plt.loglog(Pk['k'], Pk['power'].real - Pk.attrs['shotnoise'])
#plt.loglog(Pk['k'], Pk['power'].real)


# format the axes
plt.xlabel(r"$k$ [$h \ \mathrm{Mpc}^{-1}$]")
plt.ylabel(r"$P(k)$ [$h^{-3}\mathrm{Mpc}^3$]")
#plt.xlim(0.001, 0.6)
plt.xlim(0.01, 10.0)
plt.ylim(1,100000)

fig1.savefig("plot.pdf")